In [15]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [16]:
from fastbook import *

In [17]:
from fastai.vision.all import *

In [18]:
path = untar_data(URLs.IMAGENETTE)

In [19]:
dblock = DataBlock(
    blocks=(ImageBlock,CategoryBlock),
    get_items=get_image_files,
    get_y=parent_label,
    item_tfms = Resize(460),
    batch_tfms=aug_transforms(size=224,min_scale=0.75)
)

In [20]:
dls = dblock.dataloaders(path,bs=64)

In [21]:
model=xresnet50()
learn = Learner(dls,model,loss_func=CrossEntropyLossFlat(),metrics=accuracy)
learn.fit_one_cycle(5,3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.649648,2.275275,0.387603,03:26
1,1.250149,1.257275,0.619119,03:26
2,0.971736,1.249541,0.609037,03:22
3,0.739290,0.648696,0.797984,03:22
4,0.578705,0.570140,0.824869,03:21


In [22]:
def get_dls(bs,size):
  dblock = DataBlock(
      blocks=(ImageBlock,CategoryBlock),
      get_items=get_image_files,
      get_y = parent_label,
      item_tfms = Resize(460),
      batch_tfms = [*aug_transforms(size=size,min_scale=0.75),
                    Normalize.from_stats(*imagenet_stats)]
  )

  return dblock.dataloaders(path,bs=bs)

In [23]:
dls = get_dls(64,224)

In [24]:
x,y = dls.one_batch()
x.mean(dim=[0,2,3]), x.std(dim=[0,2,3])

(TensorImage([-0.1589, -0.1222, -0.0037], device='cuda:0'),
 TensorImage([1.2495, 1.2481, 1.2853], device='cuda:0'))

In [25]:
model = xresnet50()
learn = Learner(dls,model,loss_func=CrossEntropyLossFlat(), metrics=accuracy)
learn.fit_one_cycle(5,3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.629250,2.005707,0.469380,03:29
1,1.222951,1.564613,0.555639,03:20
2,0.981351,0.838514,0.736744,03:20
3,0.736297,0.683936,0.778193,03:21
4,0.600061,0.572746,0.819642,03:19


In [26]:
dls = get_dls(128,128)
learn = Learner(dls,xresnet50(),loss_func=CrossEntropyLossFlat(),metrics=accuracy)

learn.fit_one_cycle(4,3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.837376,2.511142,0.406273,02:58
1,1.295639,1.067292,0.655713,02:48
2,0.965486,0.791556,0.752427,02:51
3,0.758835,0.689909,0.782674,02:48


In [27]:
learn.dls = get_dls(64,224)
learn.fine_tune(5,1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,0.829620,1.720453,0.551531,03:17


epoch,train_loss,valid_loss,accuracy,time
0,0.649505,0.728385,0.772965,03:17
1,0.673068,0.701033,0.788275,03:16
2,0.593702,0.540507,0.832711,03:18
3,0.509065,0.506277,0.841673,03:18
4,0.453288,0.474222,0.856983,03:18


In [28]:
from fastai.callback.mixup import *
mixup = MixUp(0.5)